# Sample Steam Reviews with word-level RNN with GloVe embeddings

Code inspired from https://github.com/woctezuma/sample-steam-reviews

In [1]:
from google.colab import drive

mount_folder = '/content/gdrive'
drive.mount(mount_folder)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd '/content/gdrive/My Drive/'

/content/gdrive/My Drive


In [0]:
#!rm -rf sample-steam-reviews/
#!git clone https://github.com/woctezuma/sample-steam-reviews.git

In [4]:
%cd sample-steam-reviews/

/content/gdrive/My Drive/sample-steam-reviews


In [5]:
!git pull

Already up to date.


In [0]:
!pip install -r requirements.txt

In [0]:
!python -m spacy download en_vectors_web_lg

In [0]:
#!python word_level_rnn_with_embeddings.py

In [9]:
import spacy

from download_review_data import get_artifact_app_id
from export_review_data import get_output_file_name

from word_level_rnn_with_embeddings import train_model, get_vocabulary_file_name, generate_examples

Using TensorFlow backend.


In [0]:
app_id = get_artifact_app_id()
text_file_name = get_output_file_name(app_id)

In [0]:
!ls model.word_level_rnn_with_embeddings.*.hdf5

In [0]:
max_sentence_len = 100
overlap_size = 95
num_epochs = 20
initial_epoch = 0
full_model_filename = None # 'model.word_level_rnn_with_embeddings.epoch_{:02d}.hdf5'.format(initial_epoch)
filter_max = True

In [15]:
# Train

model, sorted_data_driven_vocabulary = train_model(path=text_file_name,
                                                   max_sentence_len=max_sentence_len,
                                                   overlap_size=overlap_size,
                                                   num_epochs=num_epochs,
                                                   full_model_filename=full_model_filename,
                                                   initial_epoch=initial_epoch,
                                                   filter_max=filter_max)

with open(get_vocabulary_file_name(), 'w', encoding='utf-8') as f:
    print(sorted_data_driven_vocabulary, file=f)


Loading GloVe...

Preparing the sentences...
Num sentences: 12331
Num unique words: 13887
Result embedding shape: (13887, 300)

Preparing the data for LSTM...
train_x shape: (12331, 100)
train_y shape: (12331,)

Training LSTM...
Epoch 1/20
12331/12331 [==============================] - 54s 4ms/step - loss: 7.1067

Generating text after epoch: 0
i like this game because...
-> i like this game because disadvantages 12ish bim advancement kidda shame variety path eat discover
i do not like this game because...
-> i do not like this game because visit refreshes opposed seamless list freely parlor 0121 replication 1017
the...
-> the too prevent saturated masterclass omfg crashes fundamentally shell accessory immediately
a...
-> a ho foresight discussion thati dem motivation more wont ratios selfserving
Sadly, randomness affects the most important aspects of gameplay. From a competitive standpoint, this game feels very much like Gwent in how alot it is a game of math. If you really want to p

In [0]:
# Show results

load_previous_vocabulary = False

if load_previous_vocabulary:
    with open(get_vocabulary_file_name(), 'r', encoding='utf-8') as f:
        sorted_data_driven_vocabulary = f.readlines()

nlp = spacy.load('en_vectors_web_lg')
word_model = nlp.vocab

generate_examples(model, sorted_data_driven_vocabulary, word_model, num_generated=10)